# Answering Business Questions using SQL

## Introduction

In this project we'll use SQL to answer business questions using the Chinook database. The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers. The structure of the database can be see [here](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg).

First we'll connect the notebook to the database:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of the Data

Let's start by getting familiar with our data, querying the database to get a list of all tables and views on our database.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * FROM customer
LIMIT 10

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


In [4]:
%%sql
SELECT * FROM invoice
LIMIT 10

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
6,31,2017-01-10 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98
7,49,2017-01-12 00:00:00,Ordynacka 10,Warsaw,None,Poland,00-358,10.89
8,59,2017-01-13 00:00:00,"3,Raj Bhavan Road",Bangalore,None,India,560001,9.9
9,18,2017-01-18 00:00:00,627 Broadway,New York,NY,USA,10012-2612,8.91
10,31,2017-01-18 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label and we've been tasked with selecting the first three albums thath will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre   |
|-------------|---------|
|   Regal     | Hip-Hop |
|   Red Tone  | Punk    |
|   Meteor and the Girls     | Pop     |
|   Slim Jim Bites     | Blues  |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [5]:
%%sql
WITH only_in_usa AS
    (
    SELECT * FROM invoice i
    INNER JOIN invoice_line il ON il.invoice_id= i.invoice_id
    WHERE billing_country = "USA"
    ),
    
    only_in_usa_with_genre AS
    (
    SELECT
        invoice_line_id,
        g.name genre
    FROM only_in_usa ou
    INNER JOIN track t ON t.track_id = ou.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    ),
    
    total_sales AS
    (
    SELECT
        COUNT(invoice_line_id)
    FROM only_in_usa_with_genre
    )
    
    
SELECT 
    genre,
    COUNT(invoice_line_id) number_of_sales,
    ROUND(CAST(COUNT(invoice_line_id) AS float)/ CAST((SELECT * FROM total_sales) AS float), 2) percentage_sales
FROM only_in_usa_with_genre ug
GROUP BY genre
ORDER BY number_of_sales DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


genre,number_of_sales,percentage_sales
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Pop,22,0.02
Latin,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


As we can see the most popular genre from our list is Blues, followed by Por and Hip-hop. Thus we recommend sign with Red Tone (Punk), Slim Jim Bites (Blues) and Meteor and the Girls (Pop).

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [6]:
%%sql
WITH sales_support_agent AS
    (
    SELECT * FROM employee
    WHERE title = "Sales Support Agent"
    ),
    
    sales_support_agent_invoice AS
    (
    SELECT * FROM sales_support_agent ssa
    LEFT JOIN customer c ON c.support_rep_id = ssa.employee_id
    LEFT JOIN invoice i ON i.customer_id = c.customer_id
    )

SELECT 
    first_name || " " || last_name sales_agent,
    ROUND(SUM(total),2) total_amount_of_sales,
    hire_date
FROM sales_support_agent_invoice
GROUP BY sales_agent
ORDER BY total_amount_of_sales DESC

 * sqlite:///chinook.db
Done.


sales_agent,total_amount_of_sales,hire_date
Jane Peacock,1731.51,2017-04-01 00:00:00
Margaret Park,1584.0,2017-05-03 00:00:00
Steve Johnson,1393.92,2017-10-17 00:00:00


We can see that Jane has more sales than Margaret and Steve but she also has more seniority than Margaret and Steve.

## Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, we have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [7]:
%%sql
WITH customer_country AS
    (
    SELECT
        c.customer_id,
        CASE
            WHEN (SELECT COUNT(*)
                 FROM customer
                 WHERE country = c.country) = 1 THEN "Other"
                 ELSE c.country
                 END AS country,
        COUNT(i.invoice_id) sales_per_customer,
        SUM(i.total) total_per_customer,
        AVG(i.total) avg_order
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY c.customer_id
    )

SELECT
    country,
    COUNT(customer_id) number_customers,
    ROUND(SUM(total_per_customer),2) total_sales,
    ROUND(SUM(total_per_customer) / COUNT(customer_id),2) avg_per_customer,
    ROUND(avg_order,2) avg_order
FROM 
    (
    SELECT
        cc.*,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
        FROM customer_country cc
    )
GROUP BY country
ORDER BY sort ASC, number_customers DESC

 * sqlite:///chinook.db
Done.


country,number_customers,total_sales,avg_per_customer,avg_order
USA,13,1040.49,80.04,9.28
Canada,8,535.59,66.95,11.11
Brazil,5,427.68,85.54,8.38
France,5,389.07,77.81,8.8
Germany,4,334.62,83.66,7.47
United Kingdom,3,245.52,81.84,8.54
Czech Republic,2,273.24,136.62,8.03
India,2,183.15,91.58,8.61
Portugal,2,185.13,92.57,7.92
Other,15,1094.94,73.0,8.03


## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [8]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674
